In [9]:
%load_ext autoreload
%autoreload 2
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import clin.llm
import clin.eval.med_status
import clin.parse
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'
from typing import List
results_dir = '../results/'
# results_dir = '../results_tmp/'
from clin.config import PATH_REPO
import datasets
import imodelsx.process_results
def viz_blue_rows(df):
    return (
        df
        # highlight rows 2 and 4 including the index
        .style.apply(lambda x: ['background: lightblue' if 'self-verified' in x.name else '' for i in x], axis=1)
        # bold index label for row 2
        .apply(lambda x: ['font-weight: bold' if 'self-verified' in x.name else '' for i in x], axis=1)
        .format(precision=3)
    )   
r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)
# r = imodelsx.process_results.average_over_seeds(r, experiment_filename=join(PATH_REPO, 'experiments', 'eval_model.py'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 42/42 [00:00<00:00, 1845.72it/s]


In [10]:
r_med = r[r.dataset_name == 'medication_status']
dset = datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')
df_val = pd.DataFrame.from_dict(dset['validation'])
df = pd.DataFrame.from_dict(dset['test'])
# df = pd.concat([val, test])
nums = np.arange(len(df)).tolist()
np.random.default_rng(seed=13).shuffle(nums)
# dfe = df.iloc[nums]
r_med = clin.eval.med_status.add_status_eval(r_med, df)

cols = {
    'f1___original': 'Medication name, original',
    'f1___ov_pv_ev': 'Medication name, self-verified',
    'status_f1_macro_cond___original': 'Medication status, original',
    'status_f1_macro_cond___sv': 'Medication status, self-verified',
}
rt_med_status = r_med.groupby(['checkpoint', 'n_shots'])[list(cols.keys())].mean().rename(columns=cols).round(3).T
viz_blue_rows(rt_med_status)

  0%|          | 0/2 [00:00<?, ?it/s]

/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r[k] = d[k]
/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r[k] = d[k]
/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [11]:
r_ebm = r[r.dataset_name == 'ebm']
cols = {
    'f1___original': 'Clinical trial arm, original',
    'f1___ov_pv_ev': 'Clinical trial arm, self-verified',
}
rt_ebm = r_ebm.groupby(['checkpoint', 'n_shots'])[list(cols.keys())].mean().rename(columns=cols).round(3).T
viz_blue_rows(rt_ebm)

In [12]:
rt = pd.concat((rt_ebm, rt_med_status))
viz_blue_rows(rt)